# Clasificación con SVM para Diagnóstico de Cáncer de Mama

Este notebook implementa un modelo de Support Vector Machine (SVM) para clasificar tumores como malignos o benignos basado en características celulares.

## Importar bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.decomposition import PCA
import pickle

## Cargar los datos

In [ ]:
# Cargar los datos
df = pd.read_csv('../data/breast-cancer.csv')

# Exploración inicial
print("Información del dataset:")
df.info()

print("\nDistribución de clases:")
print(df['diagnosis'].value_counts())

## Preparación de datos

In [ ]:
# Convertir la variable objetivo a numérica (M=1, B=0)
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

# Eliminar la columna ID que no aporta información para la predicción
df = df.drop('id', axis=1)

# Separar características y variable objetivo
X = df.drop('diagnosis', axis=1)
y = df['diagnosis']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f'Tamaño del conjunto de entrenamiento: {X_train.shape[0]} muestras, {X_train.shape[1]} características')
print(f'Tamaño del conjunto de prueba: {X_test.shape[0]} muestras, {X_test.shape[1]} características')

## Reducción de dimensionalidad con PCA

In [ ]:
# Aplicar PCA para reducir la dimensionalidad
pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Visualizar los datos después de aplicar PCA
plt.figure(figsize=(10, 8))
for i, target_name in enumerate(['Benigno', 'Maligno']):
    plt.scatter(X_train_pca[y_train == i, 0], X_train_pca[y_train == i, 1], label=target_name)
plt.title('Visualización de datos con PCA')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f'Varianza explicada por las dos componentes principales: {np.sum(pca.explained_variance_ratio_):.4f}')

## Optimización de hiperparámetros

In [ ]:
# Definir la cuadrícula de hiperparámetros
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['rbf', 'linear']
}

# Crear el modelo base
svm = SVC(probability=True, random_state=42)

# Configurar la búsqueda de cuadrícula
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Entrenar el modelo con búsqueda de cuadrícula
grid_search.fit(X_train_scaled, y_train)

# Obtener los mejores hiperparámetros
print("Mejores hiperparámetros:")
print(grid_search.best_params_)

# Obtener el mejor modelo
best_svm = grid_search.best_estimator_

## Evaluación del modelo

In [ ]:
# Realizar predicciones
y_pred = best_svm.predict(X_test_scaled)
y_prob = best_svm.predict_proba(X_test_scaled)[:, 1]

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Exactitud (Accuracy): {accuracy:.4f}')

# Informe de clasificación
print('
Informe de clasificación:')
print(classification_report(y_test, y_pred, target_names=['Benigno', 'Maligno']))

# Matriz de confusión
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Benigno', 'Maligno'], yticklabels=['Benigno', 'Maligno'])
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.savefig('../results/breast_cancer_svm_confusion_matrix.png')
plt.show()

## Curva ROC

In [ ]:
# Calcular la curva ROC
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

# Visualizar la curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Área bajo la curva = {roc_auc:.4f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.savefig('../results/breast_cancer_svm_roc_curve.png')
plt.show()

## Visualización de la frontera de decisión

In [ ]:
# Entrenar un modelo SVM con los datos reducidos por PCA
svm_pca = SVC(kernel=best_svm.kernel, C=best_svm.C, gamma=best_svm.gamma, probability=True, random_state=42)
svm_pca.fit(X_train_pca, y_train)

# Crear una malla para visualizar la frontera de decisión
h = 0.02  # Tamaño del paso en la malla
x_min, x_max = X_train_pca[:, 0].min() - 1, X_train_pca[:, 0].max() + 1
y_min, y_max = X_train_pca[:, 1].min() - 1, X_train_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

# Predecir la clase para cada punto en la malla
Z = svm_pca.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Visualizar la frontera de decisión
plt.figure(figsize=(10, 8))
plt.contourf(xx, yy, Z, alpha=0.8, cmap=plt.cm.coolwarm)

# Visualizar los puntos de entrenamiento
for i, target_name in enumerate(['Benigno', 'Maligno']):
    plt.scatter(X_train_pca[y_train == i, 0], X_train_pca[y_train == i, 1], label=target_name)

plt.title('Frontera de Decisión SVM con PCA')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend()
plt.savefig('../results/breast_cancer_svm_decision_boundary.png')
plt.show()

## Guardar el modelo

In [ ]:
# Guardar el modelo entrenado, el scaler y el PCA
with open('../models/modelo_breast_cancer_svm.pkl', 'wb') as archivo:
    pickle.dump(best_svm, archivo)

with open('../models/scaler_breast_cancer_svm.pkl', 'wb') as archivo:
    pickle.dump(scaler, archivo)

with open('../models/pca_breast_cancer_svm.pkl', 'wb') as archivo:
    pickle.dump(pca, archivo)

print('Modelo, scaler y PCA guardados correctamente.')